In [74]:
import numba as nb

In [75]:
import squidpy as sq
import scanpy as sc
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, isspmatrix_csr

In [76]:
import liana as li

In [77]:
from matplotlib.pyplot import hist

In [78]:
from liana.method._global_lr_pipe import _global_lr_pipe
from liana.method.sp._spatialdm import _get_ordered_matrix, _standardize_matrix

In [6]:
# # scHOT data
# counts = pd.read_csv("data/counts_mat.csv")
# weights = pd.read_csv("data/weight_mat.csv")
# var = pd.DataFrame(counts[['Unnamed: 0']]).set_index('Unnamed: 0')
# var.index.name = None
# adata = sc.AnnData(X=csr_matrix(counts.drop(counts.columns[0], axis=1), dtype=np.float32).T, var=var)
# adata.obsm['proximity'] = csr_matrix(weights)

In [202]:
# scHOT data test
adata = sc.read_h5ad("test_spatial.h5ad")
resource = pd.DataFrame({"ligand":["Dnm1l", "Arrb1", "Igf2", "Dnm1l"], "receptor":["Gucy1b3", "Mtor", "Tuba1a", "Fam63b"]})
dist = adata.obsm['proximity']

In [203]:
n_perm = 100
seed = 0

In [163]:
# full visium slide
# load the pre-processed dataset
img = sq.datasets.visium_hne_image()
adata = sq.datasets.visium_hne_adata()

li.mt.get_spatial_proximity(adata=adata, parameter=200, bypass_diagonal=False, cutoff=0.1)
dist = adata.obsm['proximity']

my_p = li.pl.proximity_plot(adata, idx=100)
resource = li.resource.select_resource("mouseconsensus")

In [204]:
temp, lr_res, ligand_pos, receptor_pos = _global_lr_pipe(adata=adata,
                                                         resource=resource,
                                                         expr_prop=0.05,
                                                         use_raw=False,
                                                         verbose=True,
                                                         layer=None,
                                                         _key_cols=['ligand_complex', 'receptor_complex'],
                                                         _complex_cols=['ligand_means', 'receptor_means'],
                                                         _obms_keys=['proximity'],
                                                         resource_name=None
                                                         )

Using `.X`!
Converting mat to CSR format


In [205]:
# lr_res = lr_res.head(50)

In [206]:
x_mat = _get_ordered_matrix(temp.X, ligand_pos, lr_res.ligand).A.astype(np.float32).T
y_mat = _get_ordered_matrix(temp.X, receptor_pos, lr_res.receptor).A.astype(np.float32).T

In [207]:
lr_res.head()

,interaction,ligand,receptor,ligand_complex,receptor_complex,ligand_means,ligand_props,receptor_means,receptor_props,prop_min
0,Dnm1l&Gucy1b3,Dnm1l,Gucy1b3,Dnm1l,Gucy1b3,2.682889,0.950000,3.117442,0.976923,0.950000
1,Dnm1l&Fam63b,Dnm1l,Fam63b,Dnm1l,Fam63b,2.682889,0.950000,1.781250,0.830769,0.830769
2,Arrb1&Mtor,Arrb1,Mtor,Arrb1,Mtor,2.665413,0.957692,2.298451,0.907692,0.907692
3,Igf2&Tuba1a,Igf2,Tuba1a,Igf2,Tuba1a,1.924077,0.892308,6.466562,1.000000,0.892308


In [208]:
@nb.njit(nb.f4(nb.f4[:], nb.f4[:], nb.f4[:], nb.f4, nb.boolean), cache=True)
def wcor(x, y, w, wsum, rank):
    
    if rank:
        x = np.argsort(x).argsort().astype(nb.f4)
        y = np.argsort(y).argsort().astype(nb.f4)
    
    wx = w * x
    wy = w * y
    
    numerator = wsum * np.sum(wx * y) - np.sum(wx) * np.sum(wy)
    
    denominator_x = wsum * np.sum(w * (x**2)) - np.sum(wx)**2
    denominator_y = wsum * np.sum(w * (y**2)) - np.sum(wy)**2
    denominator = np.sqrt(denominator_x * denominator_y + 1.0e-20) # avoid division by zero
    
    return numerator / denominator

In [209]:
@nb.njit(nb.f4(nb.f4[:], nb.f4[:], nb.f4[:], nb.f4, nb.int8), cache=True)
def _wcoex(x, y, w, wsum, method):
        if method == 0: # pearson
            c = wcor(x, y, w, wsum, False)
        if method == 1: # spearman
            c = wcor(x, y, w, wsum, True)
        return c

In [210]:
# 0 = pearson, 1 = spearman
@nb.njit(nb.f4[:,:](nb.f4[:,:], nb.f4[:,:], nb.f4[:,:], nb.f4, nb.int8, nb.f4), cache=True, parallel=True)
def weighted_coexpression(x_mat, y_mat, weight, wsum, method, msk_thr):
    spot_n = x_mat.shape[0]
    xy_n = x_mat.shape[1]
    
    local_correlations = np.zeros((spot_n, xy_n), dtype=nb.f4) # nb.f4
    
    for i in nb.prange(spot_n):
        w = weight[i, :]
        msk = w > msk_thr
        
        for j in range(xy_n):
            x = x_mat[:, j][msk]
            y = y_mat[:, j][msk]
            
            local_correlations[i, j] = _wcoex(x, y, w[msk], wsum, method)
    
    return local_correlations

In [211]:
dist = adata.obsm['proximity']
weight = dist.A.astype(np.float32)
wsum = np.sum(weight[0,:])

In [212]:
%%time
local_pc = weighted_coexpression(x_mat, y_mat, weight, wsum, method=0, msk_thr=0)

CPU times: user 248 ms, sys: 4 µs, total: 248 ms
Wall time: 24.1 ms


In [213]:
local_pc

array([[-1.3271085e-03,  3.9822362e-02,  2.4550273e-01,  7.1733451e-01],
       [ 1.7837617e-01,  3.2363278e-01,  4.4977826e-01,  6.3835293e-01],
       [-7.0064294e-04,  2.5916928e-01,  3.0881086e-01,  7.2371179e-01],
       ...,
       [ 1.1166877e-01,  2.4904940e-02, -6.1331499e-01,  5.1184511e-01],
       [-5.4959452e-01,  4.4820198e-01,  6.4821437e-02,  2.5075829e-01],
       [-1.9903824e-01,  3.0093941e-01,  2.7269343e-01,  7.2044134e-01]],
      dtype=float32)

In [214]:
%%time
local_sp = weighted_coexpression(x_mat, y_mat, weight, wsum, method=1, msk_thr=0)

CPU times: user 133 ms, sys: 0 ns, total: 133 ms
Wall time: 17.3 ms


In [215]:
local_sp

array([[ 0.14309415, -0.10581391,  0.22754101,  0.8548867 ],
       [ 0.05121762,  0.29360715,  0.44227427,  0.77065367],
       [ 0.25516036,  0.04407905,  0.23457415,  0.85224676],
       ...,
       [-0.06165615,  0.14110777, -0.5214374 ,  0.4505265 ],
       [-0.43638706,  0.66784775,  0.03018612,  0.38507667],
       [ 0.00177394,  0.12166166,  0.27545825,  0.8435311 ]],
      dtype=float32)

pass as fun

In [168]:
# @nb.njit(nb.f4(nb.f4[:], nb.f4[:], nb.f4[:], nb.f4, nb.boolean), cache=True)
def wcor(x, y, w, wsum, spearman):
    
    if spearman:
        x = np.argsort(x).argsort().astype(np.float32)
        y = np.argsort(y).argsort().astype(np.float32)
    
    wx = w * x
    wy = w * y
    
    numerator = wsum * np.sum(wx * y) - np.sum(wx) * np.sum(wy)
    
    denominator_x = wsum * np.sum(w * (x**2)) - np.sum(wx)**2
    denominator_y = wsum * np.sum(w * (y**2)) - np.sum(wy)**2
    denominator = np.sqrt(denominator_x * denominator_y + 1.0e-20) # avoid division by zero
    
    return numerator / denominator

In [169]:
@nb.njit(nb.f4[:,:](nb.f4[:,:], nb.f4[:,:], nb.f4[:,:], nb.f4, nb.boolean, nb.f4), cache=True, parallel=True)
def calculate_wcor(x_mat, y_mat, weight, wsum, spearman, msk_thr):
    spot_n = x_mat.shape[0]
    xy_n = x_mat.shape[1]
    
    local_correlations = np.zeros((spot_n, xy_n), dtype=nb.f4) # nb.f4
    
    for i in nb.prange(spot_n):
        w = weight[i, :]
        msk = w > msk_thr
        
        for j in range(xy_n):
            x = x_mat[:, j][msk]
            y = y_mat[:, j][msk]
            
            local_correlations[i, j] = wcor(x, y, w[msk], wsum, spearman)
    
    return local_correlations

Fully-vectorized

In [133]:
import squidpy as sq
import scanpy as sc
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, isspmatrix_csr

In [134]:
import liana as li

In [135]:
from matplotlib.pyplot import hist
from scipy.stats import spearmanr, pearsonr

In [136]:
from liana.method._global_lr_pipe import _global_lr_pipe
from liana.method.sp._spatialdm import _get_ordered_matrix, _standardize_matrix

In [137]:
from scipy.stats import rankdata

In [138]:
# ligand-receptor mats
ligand_mat = _get_ordered_matrix(temp.X, ligand_pos, lr_res.ligand).T
receptor_mat = _get_ordered_matrix(temp.X, receptor_pos, lr_res.receptor).T

In [150]:
def calculate_local_correlations(x_mat, y_mat, dist, method="pearson"):
    if method not in ["pearson", "spearman"]:
        raise ValueError("method must be one of 'pearson', 'spearman'")
    
    # transpose
    x_mat, y_mat = x_mat.T, y_mat.T
    
    weight = dist.A.T
    weight_sums = np.sum(weight, axis = 0).flatten()
        
    if method=="spearman":
        x_mat = rankdata(x_mat, axis=1)
        y_mat = rankdata(y_mat, axis=1)
    
    # standard pearson
    n1 = (((x_mat * y_mat).dot(weight)) * weight_sums)
    n2 = (x_mat.dot(weight)) * (y_mat.dot(weight))
    numerator = n1 - n2
    
    denominator_x = (weight_sums * (x_mat ** 2).dot(weight)) - (x_mat.dot(weight))**2
    denominator_y = (weight_sums * (y_mat ** 2).dot(weight)) - (y_mat.dot(weight))**2
    denominator = np.sqrt(denominator_x * denominator_y)
    denominator[denominator == 0] = np.finfo(np.float32).eps # add noise to avoid division by zero
    
    local_corrs = (numerator / denominator)
    
    return local_corrs

In [151]:
dist = adata.obsm['proximity']

In [152]:
local_pc = calculate_local_correlations(x_mat = ligand_mat.A, y_mat=receptor_mat.A, dist=dist, method="pearson")

In [153]:
local_sp = calculate_local_correlations(x_mat = ligand_mat.A, y_mat=receptor_mat.A, dist=dist, method="spearman")

In [154]:
local_pc

array([[-1.32974227e-03,  1.78378213e-01, -6.99047437e-04, ...,
         1.11669926e-01, -5.49594345e-01, -1.99038894e-01],
       [ 3.98228109e-02,  3.23634089e-01,  2.59169939e-01, ...,
         2.49048467e-02,  4.48201488e-01,  3.00939823e-01],
       [ 2.45500324e-01,  4.49778688e-01,  3.08810443e-01, ...,
        -6.13314098e-01,  6.48224085e-02,  2.72694903e-01],
       [ 7.17321830e-01,  6.38337093e-01,  7.23703539e-01, ...,
         5.11895209e-01,  2.50760443e-01,  7.20444154e-01]])

In [155]:
local_sp

array([[ 0.02745524,  0.16780172,  0.02950179, ...,  0.05846293,
        -0.47176155, -0.16618834],
       [ 0.06676514,  0.38843465,  0.29575594, ...,  0.05419157,
         0.48301053,  0.33821676],
       [ 0.25214086,  0.51057389,  0.31465863, ..., -0.57736091,
        -0.06078281,  0.33525634],
       [ 0.85249697,  0.7571208 ,  0.85231423, ...,  0.40853417,
         0.36100299,  0.87528304]])

In [ ]:
from liana.method.sp._spatial_utils import _local_permutation_pvals

In [ ]:
local_pvals = _local_permutation_pvals(x_mat = ligand_mat.A, 
                                       y_mat = receptor_mat.A, 
                                       local_truth=local_pc,
                                       local_fun=calculate_local_correlations,
                                       dist=dist, 
                                       n_perm=n_perm, 
                                       positive_only=False,
                                       seed=seed)

In [ ]:
local_pvals

In [ ]:
local_sp = calculate_local_correlations(x_mat = ligand_mat.A, y_mat=receptor_mat.A, dist=dist, method="spearman")

In [ ]:
local_sp_pvals = _local_permutation_pvals(x_mat = ligand_mat.A, 
                                          y_mat = receptor_mat.A, 
                                          local_truth=local_sp,
                                          local_fun=calculate_local_correlations,
                                          dist=dist, 
                                          n_perm=n_perm, 
                                          positive_only=False,
                                          seed=seed,
                                          method="spearman"
                                          )

In [ ]:
local_pvals.shape

In [ ]:
local_sp_pvals.shape

In [ ]:
spearmanr(local_sp_pvals[1,:], local_pvals[1,:])

Global summary of the local scores:

In [ ]:
lr_res.loc[:,['pearson_mean','pearson_sd']] = np.vstack([np.mean(local_pc, axis=1), np.std(local_pc, axis=1)]).T

In [ ]:
lr_res.sort_values(by='pearson_mean', ascending=False)

masked

In [ ]:
# ligand-receptor mats
ligand_mat = _get_ordered_matrix(temp.X, ligand_pos, lr_res.ligand)
receptor_mat = _get_ordered_matrix(temp.X, receptor_pos, lr_res.receptor)

In [ ]:
import scipy.stats as stats

In [ ]:
def masked_wcor(x, y, weight, method='spearman_nzw'):
    spot_n = x.shape[0]
    
    # reshape x and y to be the same shape as weight
    x = np.reshape(np.repeat(x, spot_n), newshape=(spot_n, spot_n)).T
    y = np.reshape(np.repeat(y, spot_n), newshape=(spot_n, spot_n)).T
    
    # mask x and y with the same mask as weight
    x_masked = np.ma.array(x, mask = weight.mask, fill_value=np.nan)
    y_masked = np.ma.array(y, mask = weight.mask, fill_value=np.nan)
    
    if method == 'spearman_nzw':
        x_masked = stats.mstats.rankdata(x_masked, axis=1)
        y_masked = stats.mstats.rankdata(y_masked, axis=1)
        
    
    # calculate weighted pearson correlation
    wsum = np.ma.sum(weight, axis=1)
    xws = np.ma.sum(weight * x_masked, axis=1)
    yws = np.ma.sum(weight * y_masked, axis=1)
    
    n1 = wsum * np.ma.sum(weight * x_masked * y_masked, axis=1)
    n2 = xws * yws
    numerator = n1 - n2
    
    denominator_x = wsum * np.ma.sum(weight * (x_masked**2), axis=1) - xws**2
    denominator_y = wsum * np.ma.sum(weight * (y_masked**2), axis=1) - yws**2
    wcor = numerator / np.ma.sqrt(denominator_x * denominator_y)
    
    return wcor.data

In [ ]:
def calculate_masked_correlations(x_mat, y_mat, dist, method='spearman_nzw'):
    weight = dist.A
    msk = np.logical_not(weight>0).astype(np.int16)
    weight = np.ma.masked_array(weight, mask=msk)
    # calculate for each x and y combination
    local_correlations = []
    
    for i in range(x_mat.shape[0]):
        local_correlations.append(masked_wcor(x_mat[i, :], y_mat[i, :], weight=weight))
    local_correlations = np.array(local_correlations)
    
    return local_correlations


In [ ]:
ligand_mat.shape

In [ ]:
%%time

dist = adata.obsm['proximity']

masked_sp = calculate_masked_correlations(ligand_mat[0:10,:].A, receptor_mat[0:10,:].A, dist=dist, method='spearman_nzw')

In [ ]:
masked_sp[0:10,0:10]

In [ ]:
hist(masked_sp[0,:])

In [ ]:
spot_n = x.shape[1]

In [ ]:
weight = dist.A
msk = np.logical_not(weight>0).astype(np.int16)
weight = np.ma.masked_array(weight, mask=msk)

In [ ]:
x, y = ligand_mat[0,:].A, receptor_mat[0,:].A

In [ ]:
spot_n


In [ ]:
# reshape x and y to be the same shape as weight
x = np.reshape(np.repeat(x, spot_n), newshape=(spot_n, spot_n)).T
y = np.reshape(np.repeat(y, spot_n), newshape=(spot_n, spot_n)).T

In [ ]:
# mask x and y with the same mask as weight
x_masked = np.ma.array(x, mask = weight.mask, fill_value=np.nan)
y_masked = np.ma.array(y, mask = weight.mask, fill_value=np.nan)

In [ ]:
x_masked = stats.mstats.rankdata(x_masked, axis=1)
y_masked = stats.mstats.rankdata(y_masked, axis=1)

local p-values

In [ ]:
from numpy import random
from tqdm import tqdm

In [ ]:
rng = random.default_rng(0)
n_perm = 100
positive_only = True # remove this option?

In [ ]:
dist = adata.obsm['proximity']

In [ ]:
local_pc.shape

In [ ]:
def _get_local_permutation_pvals(x_mat, y_mat, local_truth, local_fun, dist, n_perm, positive_only=True, **kwargs):
    xy_n = local_truth.shape[0]
    spot_n = local_truth.shape[1]
    
    print(spot_n)
    
    # permutation cubes to be populated
    local_pvals = np.zeros((xy_n, spot_n))
    
    # shuffle the matrix
    for i in tqdm(range(n_perm)):
        _idx = rng.permutation(spot_n)
        perm_score = local_fun(x_mat = x_mat[_idx, :], y_mat=y_mat, dist=dist, **kwargs) ## TODO switch to shuffle rows, not columns
        if positive_only:
            local_pvals += np.array(perm_score >= local_truth, dtype=int)
        else:
            local_pvals += (np.array(np.abs(perm_score) >= np.abs(local_truth), dtype=int))

    local_pvals = local_pvals / n_perm
    
    return local_pvals
    

In [ ]:
from liana.method.sp._spatial_utils import _local_permutation_pvals

In [ ]:
local_pvals = _local_permutation_pvals(x_mat = ligand_mat.A, 
                                       y_mat = receptor_mat.A, 
                                       local_truth=local_pc,
                                       local_fun=calculate_local_correlations,
                                       dist=dist, 
                                       n_perm=n_perm, 
                                       positive_only=False,
                                       seed=0)

In [ ]:
local_pvals

In [ ]:
local_pvals

In [ ]:
from  scipy.sparse import csr_matrix

In [ ]:
local_pvals.shape

In [ ]:
local_pvals

In [ ]:
local_pc.shape

In [ ]:
local_pvals.shape

In [ ]:
local_masked_pvals = _get_local_permutation_pvals(x_mat = ligand_mat.A,
                                                  y_mat = receptor_mat.A,
                                                  local_truth = masked_sp,
                                                  local_fun=calculate_masked_correlations,
                                                  dist=dist,
                                                  n_perm=n_perm,
                                                  positive_only=False)

In [ ]:
local_masked_pvals

In [ ]:
local_masked_pvals.shape

In [ ]:
from scipy.stats import spearmanr,  pearsonr

In [ ]:
pearsonr(local_masked_pvals[0,:], local_pvals[0,:])